# Рубежный контроль №2
## Фруктин А.Е. РТ5-61Б Вариант №22
Метод 1 - Дерево решений

Метод 2 - Градиентный бустинг

Импортируем необходимые библиотеки

In [157]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [158]:

df = pd.read_csv('Historical Presidents Physical Data (More).csv')

In [159]:
df.head(10)

,order,name,height_cm,height_in,weight_kg,weight_lb,body_mass_index,body_mass_index_range,birth_day,birth_month,...,term_begin_year,term_begin_date,term_end_day,term_end_month,term_end_year,term_end_date,presidency_begin_age,presidency_end_age,political_party,corrected_iq
0,1,George Washington,188,74.0,79.4,175,22.5,Normal,22,2,...,1789,30-04-1789,4.0,3.0,1797.0,04-03-1797,57,65.0,Unaffiliated,140.0
1,2,John Adams,170,67.0,83.9,185,29.0,Overweight,30,10,...,1797,04-03-1797,4.0,3.0,1801.0,04-03-1801,61,65.0,Federalist,155.0
2,3,Thomas Jefferson,189,74.5,82.1,181,23.0,Normal,13,4,...,1801,04-03-1801,4.0,3.0,1809.0,04-03-1809,57,65.0,Democratic-Republican,160.0
3,4,James Madison,163,64.0,55.3,122,20.8,Normal,16,3,...,1809,04-03-1809,4.0,3.0,1817.0,04-03-1817,57,65.0,Democratic-Republican,160.0
4,5,James Monroe,183,72.0,85.7,189,25.6,Overweight,28,4,...,1817,04-03-1817,4.0,3.0,1825.0,04-03-1825,58,66.0,Democratic-Republican,139.0
5,6,John Quincy Adams,171,67.5,92.1,203,31.5,Obese,11,7,...,1825,04-03-1825,4.0,3.0,1829.0,04-03-1829,57,61.0,Democratic-Republican,175.0
6,7,Andrew Jackson,185,73.0,63.5,140,18.6,Normal,15,3,...,1829,04-03-1829,4.0,3.0,1837.0,04-03-1837,61,69.0,Democrat,145.0
7,8,Martin Van Buren,168,66.0,74.4,164,26.4,Overweight,5,12,...,1837,04-03-1837,4.0,3.0,1841.0,04-03-1841,54,58.0,Democrat,146.0
8,9,William Henry Harrison,173,68.0,73.5,162,24.6,Normal,9,2,...,1841,04-03-1841,4.0,4.0,1841.0,04-04-1841,68,68.0,Whig,146.0
9,10,John Tyler,183,72.0,72.6,160,21.7,Normal,29,3,...,1841,04-04-1841,4.0,3.0,1845.0,04-03-1845,51,54.0,Whig,148.0


In [160]:
# Удаление строк, содержащих пустые значения
df = df.dropna(axis=0, how='any')
df.isnull().sum()

order                    0
name                     0
height_cm                0
height_in                0
weight_kg                0
weight_lb                0
body_mass_index          0
body_mass_index_range    0
birth_day                0
birth_month              0
birth_year               0
birth_date               0
birthplace               0
birth_state              0
death_day                0
death_month              0
death_year               0
death_date               0
death_age                0
astrological_sign        0
term_begin_day           0
term_begin_month         0
term_begin_year          0
term_begin_date          0
term_end_day             0
term_end_month           0
term_end_year            0
term_end_date            0
presidency_begin_age     0
presidency_end_age       0
political_party          0
corrected_iq             0
dtype: int64

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 39
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order                  39 non-null     object 
 1   name                   39 non-null     object 
 2   height_cm              39 non-null     int64  
 3   height_in              39 non-null     float64
 4   weight_kg              39 non-null     float64
 5   weight_lb              39 non-null     int64  
 6   body_mass_index        39 non-null     float64
 7   body_mass_index_range  39 non-null     object 
 8   birth_day              39 non-null     int64  
 9   birth_month            39 non-null     int64  
 10  birth_year             39 non-null     int64  
 11  birth_date             39 non-null     object 
 12  birthplace             39 non-null     object 
 13  birth_state            39 non-null     object 
 14  death_day              39 non-null     float64
 15  death_month  

Сделаем label encode

In [162]:
label_encoder = LabelEncoder()
df['body_mass_index_range'] = label_encoder.fit_transform(df['body_mass_index_range'])
df['political_party'] = label_encoder.fit_transform(df['political_party'])
df = df.drop(columns=["order", "name", "birth_date", "birthplace", "birth_state", "death_date", "astrological_sign", "term_begin_date", "term_end_date"])

Разделим выборку на обучающую и тренировочную

In [163]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('political_party',axis=1),df['political_party'],test_size=0.2, random_state=42)

Обучим требуемые модели

In [164]:
model = DecisionTreeClassifier(random_state=3)
model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=3)

In [165]:
gradient_boosting_classifier = GradientBoostingClassifier()
gradient_boosting_classifier.fit(X_train, y_train)

GradientBoostingClassifier()

## Древо решений

In [166]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
print(f'Точность: {accuracy}')
print(f'precision: {precision}')
print(f'Полнота: {recall}')

Точность: 0.25
precision: 0.08
Полнота: 0.2


## Градиентный бустинг

In [167]:
y_grad_pred = gradient_boosting_classifier.predict(X_test)
grad_accuracy = accuracy_score(y_test, y_grad_pred)
grad_recall = recall_score(y_test, y_grad_pred, average='macro', zero_division=0)
grad_precision = precision_score(y_test, y_grad_pred, average='macro', zero_division=0)
print(f'Точность: {grad_accuracy}')
print(f'precision: {grad_precision}')
print(f'Полнота: {grad_recall}')

Точность: 0.125
precision: 0.1
Полнота: 0.1


Градиентный бустинг имеет более низкую точность, чем дерево решений (12,5% против 25%), в случае с precision градиентный бустинг побеждает - 10% против 8%. Однако полнота градиентного бустинга 10%? тогда как у древа 20%. Дерево решений имеет более высокую точность и более высокую полноту.

### Древо решений предпочтительнее